In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/saved_model.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/keras_metadata.pb
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/assets/vocab.txt
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.index
/kaggle/input/bert/tensorflow2/en-uncased-preprocess/3/variables/variables.data-00000-of-00001
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/saved_model.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/keras_metadata.pb
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/assets/vocab.txt
/kaggle/input/bert/tensorflow2/bert-en-uncased-l-12-h-768-a-12/2/variables/variables.index
/kaggle/input/bert/tensorflo

In [2]:
import tensorflow_text as text
import tensorflow_hub as hub
import tensorflow as tf
import random
import tqdm
from tqdm import tqdm
import copy

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
train_essays_01_source = pd.read_csv('/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv')

#Slicing the dataset with different labels
train_essays_01_source_label_1 = train_essays_01_source[train_essays_01_source['label'] == 1].reset_index(drop = True)  
train_essays_01_source_label_0 = train_essays_01_source[train_essays_01_source['label'] == 0].reset_index(drop = True)

#Set number of data points required from each label
#Creating new data set with data sampled from each labels
label_1_count = 1638 #1638 data points available
label_0_count = 13712 #13712 data points available

essay_dataset_raw = pd.concat([train_essays_01_source_label_1.sample(n= label_1_count, random_state = 36, axis = 0), 
                         train_essays_01_source_label_0.sample(n= label_0_count, random_state = 36, axis = 0)], 
                        axis = 0, sort = True).sample(frac = 1)

In [4]:
#Bert encoder

text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-12-h-768-a-12/versions/2",trainable=False)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

In [5]:
encoder_model = tf.keras.Model(inputs=text_input, outputs=sequence_output)

In [6]:
%%time
batch_size = 100
num_batches = len(essay_dataset_raw)//batch_size

essay_dataset_raw_batches = np.array_split(essay_dataset_raw.iloc[:,1], num_batches)
essay_dataset_raw_batch_list = []
i = 0

for batch in tqdm(essay_dataset_raw_batches):
    i+=1
    encoded_essay_batch = encoder_model(batch)
    essay_dataset_raw_batch_list.append(encoded_essay_batch)
    
    if i%100 == 0:
        encoded_essay = np.concatenate(essay_dataset_raw_batch_list, axis = 0)
        del essay_dataset_raw_batch_list
        essay_dataset_raw_batch_list = [encoded_essay]
    
    
encoded_essay = tf.convert_to_tensor(np.concatenate(essay_dataset_raw_batch_list, axis = 0))

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:57: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 153/153 [02:07<00:00,  1.20it/s]


CPU times: user 2min 25s, sys: 31.2 s, total: 2min 56s
Wall time: 2min 19s


In [7]:
del essay_dataset_raw_batches
del essay_dataset_raw_batch_list

In [8]:
#Train:CV:Test :: i:j:k 
i, j, k = 40, 30, 1

fraction_size = len(encoded_essay)//(i+j+k)
encoded_essay_train, encoded_essay_CV, encoded_essay_test = tf.split(encoded_essay, [fraction_size*i, fraction_size*j, len(encoded_essay)-fraction_size*(i+j)], 0)

label_train, label_CV, label_test = tf.split(essay_dataset_raw['label'], [fraction_size*i, fraction_size*j, len(essay_dataset_raw)-fraction_size*(i+j)], 0)

In [9]:
import tensorflow as tf
from tensorflow.keras import layers

# Import your input embeddings
input_embeddings = tf.keras.Input(shape=(128, 768, ))

# Apply pooling right after the input layer
pooled_input = layers.GlobalMaxPooling1D()(input_embeddings)

# Reshape pooled input to add singleton dimension
query = tf.expand_dims(pooled_input, axis=1)

# Multi-Head Attention layer with pooling layer output as key and input embedding vectors as values
num_heads = 12
key_dim = 64  # Adjust based on experimentation
attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)(query, input_embeddings)

# Global average pooling
global_pool = layers.GlobalAveragePooling1D()(attention_output)

# Dropout to prevent overfitting
dropout1 = layers.Dropout(0.2)(global_pool)

# Dense output layer with 1 unit and relu activation
dense1 = layers.Dense(320, activation="relu")(dropout1)

# Dropout to prevent overfitting
dropout2 = layers.Dropout(0.1)(dense1)

# Dense output layer with 1 unit and relu activation
dense2 = layers.Dense(64, activation="relu")(dropout2)

# Dense output layer with 1 unit and sigmoid activation (for binary classification)
output = layers.Dense(1, activation="sigmoid")(dense2)

# Create the model
model = tf.keras.Model(inputs=input_embeddings, outputs=output)


In [10]:
model.compile(
    loss = 'binary_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy', 'AUC']
)

In [11]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 128, 768)]           0         []                            
                                                                                                  
 global_max_pooling1d (Glob  (None, 768)                  0         ['input_2[0][0]']             
 alMaxPooling1D)                                                                                  
                                                                                                  
 tf.expand_dims (TFOpLambda  (None, 1, 768)               0         ['global_max_pooling1d[0][0]']
 )                                                                                                
                                                                                            

In [12]:
h = model.fit(
   encoded_essay_train, label_train,
   validation_data = (encoded_essay_CV, label_CV),
   epochs = 20,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor = 'val_auc', patience = 1, mode = 'max', 
                                      restore_best_weights = True, min_delta = 0.01, start_from_epoch = 2)
   ]
)

Epoch 1/20
270/270 [==============================] - 11s 25ms/step - loss: 0.0995 - accuracy: 0.9686 - auc: 0.9743 - val_loss: 0.0583 - val_accuracy: 0.9810 - val_auc: 0.9923
Epoch 2/20
270/270 [==============================] - 4s 17ms/step - loss: 0.0644 - accuracy: 0.9823 - auc: 0.9834 - val_loss: 0.0621 - val_accuracy: 0.9765 - val_auc: 0.9921
Epoch 3/20
270/270 [==============================] - 4s 17ms/step - loss: 0.0350 - accuracy: 0.9900 - auc: 0.9924 - val_loss: 0.0217 - val_accuracy: 0.9926 - val_auc: 0.9978
Epoch 4/20
270/270 [==============================] - 5s 17ms/step - loss: 0.0263 - accuracy: 0.9922 - auc: 0.9954 - val_loss: 0.0210 - val_accuracy: 0.9934 - val_auc: 0.9979


In [13]:
# test_set_predictions = model.predict(encoded_essay_test)
# _ = model.evaluate(encoded_essay_test, label_test)

In [14]:
test_data = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')

In [15]:
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize


# Function to split text into sequences of 128 words
def split_text_into_sequences(text):
    words = word_tokenize(text)
    sequences = [words[i:i + 128] for i in range(0, len(words), 128)]
    return [' '.join(seq) for seq in sequences]


# Apply the function to the 'text' column
test_data['text_sequences'] = test_data['text'].apply(split_text_into_sequences)

# Explode the sequences to create rows for each sequence
df_exploded = test_data.explode('text_sequences')

# Drop the original 'text' column
df_exploded = df_exploded.drop('text', axis=1)

#Resetting test_data
test_data = df_exploded.copy()
del df_exploded



In [16]:
%%time
# encoded_test_data = encoder_model(train_essays_01_source.iloc[:10000, 0])
# test_set_predictions = model.predict(encoded_test_data)


if len(test_data)<100:
    encoded_test_data = encoder_model(test_data.iloc[:, 2])
    test_set_predictions = model.predict(encoded_test_data)

else:
    batch_size = 100
    num_batches = len(test_data)//batch_size

    test_data_batches = np.array_split(test_data, num_batches)
    test_set_predictions_batch_list = []

    for batch in tqdm(test_data_batches):
        encoded_test_data_batch = encoder_model(batch.iloc[:, 2])
        test_set_predictions_batch = model.predict(encoded_test_data_batch)
        test_set_predictions_batch_list.append(test_set_predictions_batch)

    test_set_predictions = np.concatenate(test_set_predictions_batch_list, axis = 0)

1/1 [==============================] - 0s 118ms/step
CPU times: user 183 ms, sys: 7.16 ms, total: 191 ms
Wall time: 177 ms


In [17]:
test_data['prediction'] = test_set_predictions.flatten()

test_data_agg = test_data.groupby(['id','prompt_id']).aggregate({'prediction':'mean'})

test_data_agg.reset_index(inplace = True)

test_data_agg.rename(columns = {'prediction':'genarated'}, inplace = True)

In [18]:
test_data_agg['genarated'] = np.round(test_data_agg['genarated'], decimals = 4)
submission = test_data_agg[['id', 'genarated']]
submission.to_csv('/kaggle/working/submission.csv', index = False, header = submission.columns.tolist())

# EDA

In [19]:
# import matplotlib.pyplot as plt
# import numpy
# from sklearn import metrics

# def plot_confusion_matrix(label_test, test_set_predictions, threashold = 0.5):
    
#     if not test_set_pred_labels:
#         test_set_pred_labels = [prob >= threashold for prob in test_set_predictions.flatten()]

#     confusion_matrix = metrics.confusion_matrix(label_test, test_set_pred_labels)

#     cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix = confusion_matrix, display_labels = [False, True])

#     cm_display.plot()
#     plt.show()

In [20]:
# plot_confusion_matrix(label_test, test_set_predictions, 0.5)

In [21]:
# import numpy as np
# from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt
# import plotly.graph_objects as go
# from sklearn.metrics import roc_curve

# def plot_roc_curve(true_y, y_prob):
#     """
#     Plots an interactive ROC curve based on the probabilities and displays thresholds on hover.
#     """
    
#     fpr, tpr, thresholds = roc_curve(true_y, y_prob, drop_intermediate=False)

#     fig = go.Figure()

#     fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name='ROC Curve', hovertext=[f'Threshold: {threshold:.2f}' for threshold in thresholds]))

#     fig.update_layout(
#         title='ROC Curve',
#         xaxis=dict(title='False Positive Rate'),
#         yaxis=dict(title='True Positive Rate'),
#         hovermode='closest'
#     )

#     fig.show()

In [22]:
# plot_roc_curve(label_test, test_set_predictions.flatten())
#print('ROC AUC Score: ',roc_auc_score(label_test, test_set_predictions.flatten()))

# TEST Data Predctions, a different approach!

In [23]:
# test_data.info()

In [24]:
# test_data = pd.read_csv('/kaggle/input/daigt-proper-train-dataset/train_drcat_04.csv')
# #test_data.drop(columns = 'generated', inplace = True)

In [25]:
# test_data_label = test_data[['essay_id','text']].copy()
# test_data.drop(columns = ['source', 'prompt','fold'], inplace = True)
# test_data.rename(columns = {'essay_id':'id'}, inplace = True)
# test_data['prompt_id'] = test_data['id']

In [26]:
# len(test_data.iloc[6,1].split())
# #test_data.iloc[6,1]

In [27]:
# test_data.info()
# test_data = test_data.sample(frac = 0.25)

In [28]:
# test_data.info()

In [29]:
# from nltk.tokenize import word_tokenize
# from nltk import sent_tokenize


# # Function to split text into sequences of 128 words
# def split_text_into_sequences(text):
#     words = word_tokenize(text)
#     sequences = [words[i:i + 128] for i in range(0, len(words), 128)]
#     return [' '.join(seq) for seq in sequences]


# # Apply the function to the 'text' column
# test_data['text_sequences'] = test_data['text'].apply(split_text_into_sequences)

# # Explode the sequences to create rows for each sequence
# df_exploded = test_data.explode('text_sequences')

# # Drop the original 'text' column
# df_exploded = df_exploded.drop('text', axis=1)

# #Resetting test_data
# test_data = df_exploded.copy()
# del df_exploded





In [30]:
# test_data = test_data.sample(frac = 8, replace = True)

In [31]:
# %%time
# tqdm
# # Apply the function to the 'text' column
# test_data['text_sequences'] = test_data['text'].apply(split_text_into_sequences)

# # Explode the sequences to create rows for each sequence
# df_exploded = test_data.explode('text_sequences')

# # Drop the original 'text' column
# df_exploded = df_exploded.drop('text', axis=1)

# # Display the resulting dataframe
# print(df_exploded)

In [32]:
# test_data = df_exploded

In [33]:
# %%time
# # encoded_test_data = encoder_model(train_essays_01_source.iloc[:10000, 0])
# # test_set_predictions = model.predict(encoded_test_data)


# if len(test_data)<100:
#     encoded_test_data = encoder_model(test_data.iloc[:, 3])
#     test_set_predictions = model.predict(encoded_test_data)

# else:
#     batch_size = 100
#     num_batches = len(test_data)//batch_size

#     test_data_batches = np.array_split(test_data, num_batches)
#     test_set_predictions_batch_list = []

#     for batch in test_data_batches:
#         encoded_test_data_batch = encoder_model(batch.iloc[:, 3])
#         test_set_predictions_batch = model.predict(encoded_test_data_batch)
#         test_set_predictions_batch_list.append(test_set_predictions_batch)

#     test_set_predictions = np.concatenate(test_set_predictions_batch_list, axis = 0)

In [34]:
# batch.iloc[:, 3]

In [35]:
# test_set_predictions.flatten()

In [36]:
# test_data['prediction'] = test_set_predictions.flatten()

In [37]:
# test_data.head()

In [38]:
# test_data['prediction'] = test_set_predictions.flatten()

# test_data_agg = test_data.groupby(['id','prompt_id','label']).aggregate({'prediction':['max', 'min', 'mean', 'first']})

# test_data_agg.reset_index(inplace = True)

In [39]:
# test_data_agg.reset_index(inplace = True)

In [40]:
# pred_custom = []
# for index,row in test_data_agg.iterrows():
    
#     if row['prediction']['first'] >= 0.5:
#         pred_custom.append(row['prediction']['max'])
#     else:
#         pred_custom.append(row['prediction']['min'])
    

In [41]:
# test_data_agg['custom'] = pred_custom

In [42]:
# test_data_agg.head()

In [43]:
#print('ROC AUC Score: ',roc_auc_score(label_test, test_set_predictions.flatten()))

In [44]:
# for i in range(3,8):
#     print('ROC with metric:',test_data_agg.columns[i])
#     print('ROC AUC Score: ',roc_auc_score(test_data_agg.iloc[:,2], test_data_agg.iloc[:,i]))

In [45]:
# (0.9824285407738714 - 0.9667853753352559)*100